In [1]:
import os
import torch
import random
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from utils.utils import load_parameters, stack_tensors
from datasets.datasets import load_data
from inner_maximizers.inner_maximizers import inner_maximizer
from nets.ff_classifier import build_ff_classifier
import losswise
import time
import json
import numpy as np

In [2]:
parameters = load_parameters("parameters.ini")
is_cuda = eval(parameters["general"]["is_cuda"])
if is_cuda:
    os.environ["CUDA_VISIBLE_DEVICES"] = parameters["general"]["gpu_device"]

assertion_message = "Set this flag off to train models."
assert eval(parameters['dataset']['generate_feature_vector_files']) is False, assertion_message

log_interval = int(parameters["general"]["log_interval"])
num_epochs = int(parameters["hyperparam"]["ff_num_epochs"])
is_losswise = eval(parameters["general"]["is_losswise"])
is_synthetic_dataset = eval(parameters["general"]["is_synthetic_dataset"])

training_method = parameters["general"]["training_method"]
evasion_method = parameters["general"]["evasion_method"]
experiment_suffix = parameters["general"]["experiment_suffix"]
experiment_name = "[training:%s|evasion:%s]_%s" % (training_method, evasion_method,
                                                   experiment_suffix)

print("Training Method:%s, Evasion Method:%s" % (training_method, evasion_method))

seed_val = int(parameters["general"]["seed"])

random.seed(seed_val)
torch.manual_seed(seed_val)
np.random.seed(seed_val)


Training Method:rfgsm_k, Evasion Method:natural


In [3]:
evasion_iterations = int(parameters['hyperparam']['evasion_iterations'])

save_every_epoch = eval(parameters['general']['save_every_epoch'])

train_model_from_scratch = eval(parameters['general']['train_model_from_scratch'])
load_model_weights = eval(parameters['general']['load_model_weights'])
model_weights_path = parameters['general']['model_weights_path']

In [4]:
train_dataloader_dict, valid_dataloader_dict, test_dataloader_dict, num_features = load_data(
    parameters)

Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Deskto

Starting data loading
Creating import to index mapping
-- accessing file: ./helper_files/N5000/pmal/
-- accessing file: ./helper_files/N5000/pben/
Malicious files: 500
Benign files: 500
Total number of files: 1000
0 Time: 0.0  seconds
100 Time: 0.08813333511352539  seconds


Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Deskto

200 Time: 0.09472346305847168  seconds
300 Time: 0.12267112731933594  seconds
400 Time: 0.12614107131958008  seconds


Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Deskto

...

  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 wo

700 Time: 0.10363340377807617  seconds
800 Time: 0.09574246406555176  seconds
900 Time: 0.08378219604492188  seconds


Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Deskto

-- accessing file: ./helper_files/None.txt
-- accessing file: ./helper_files/N5000/pmal/
-- accessing file: ./helper_files/N5000/pben/
Malware Files: 450
Benign Files: 450
Preparing training datasets
Preparing validation datasets
Preparing testing datasets
FAL
./helper_files/N5000/pmal/Pickle_malware198.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./helper_files/N5000/pben/Pickle_benign479.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./helper_files/N5000/pmal/Pickle_malware5.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./helper_files/N5000/pben/Pickle_benign204.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./helper_files/N5000/pmal/Pickle_malware5.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./helper_files/N5000/pben/Pickle_benign479.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./helper_files/N5000/pmal/Pickle_malware198.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./helper_files/N5000/pben/Pickle_benign204.pt
FV
tensor([0., 0., 0.,  ..., 0., 0., 0.])
FAL
./h

AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Traceback (most recent call last):
  File "C:\Users\Pitch\Desktop\Sem8\Frameworks Pitch\March 19 work\datasets\datasets.py", line 72, in create_import_to_index_mapping
    lib.name.lower() + ':' + e.name for lib in binary.imports for e in lib.entries
AttributeError: 'NoneType' object has no attribute 'imports'
Tra

In [5]:

num_samples = len(train_dataloader_dict["malicious"].dataset)


In [6]:
model = build_ff_classifier(
        input_size=num_features,
        hidden_1_size=int(parameters["hyperparam"]["ff_h1"]),
        hidden_2_size=int(parameters["hyperparam"]["ff_h2"]),
        hidden_3_size=int(parameters["hyperparam"]["ff_h3"]))
model

Sequential(
  (0): Linear(in_features=2948, out_features=300, bias=True)
  (1): ReLU()
  (2): Linear(in_features=300, out_features=300, bias=True)
  (3): ReLU()
  (4): Linear(in_features=300, out_features=300, bias=True)
  (5): ReLU()
  (6): Linear(in_features=300, out_features=2, bias=True)
  (7): LogSoftmax(dim=1)
)

In [7]:
import torch.nn.functional as F

def get_beta(batch_idx, m, beta_type, epoch, num_epochs):
    if beta_type == "Blundell":
        beta = 2 ** (m - (batch_idx + 1)) / (2 ** m - 1)
    elif beta_type == "Soenderby":
        beta = min(epoch / (num_epochs // 4), 1)
    elif beta_type == "Standard":
        beta = 1 / m
    else:
        beta = 0
    return beta

def elbo(out, y, kl_sum, beta):
    ce_loss = F.cross_entropy(out, y)
    return ce_loss + beta * kl_sum

In [8]:
loss_fct = nn.NLLLoss(reduce=False)
optimizer = optim.Adam(model.parameters(), lr=float(parameters["hyperparam"]["ff_learning_rate"]))
optimizer

C:\ProgramData\Anaconda3\envs\nn_mal_2\lib\site-packages\torch\nn\_reduction.py:44: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)

In [9]:
NumCount = 0
Visited = []

In [10]:
def train(epoch):
    model.train()
    total_correct = 0.
    total_loss = 0.
    total = 0.

    current_time = time.time()

    if is_synthetic_dataset:
        # since generation of synthetic data set is random, we'd like them to be the same over epochs
        torch.manual_seed(seed_val)
        random.seed(seed_val)
    print("dataloaderr",train_dataloader_dict["benign"].dataset, train_dataloader_dict["malicious"].dataset)
    for batch_idx, ((bon_x, bon_y), (mal_x, mal_y)) in enumerate(
            zip(train_dataloader_dict["benign"], train_dataloader_dict["malicious"])):
        # Check for adversarial learning
        print("This is bacth_idx",batch_idx)
#         print(bon_x)
#         print(bon_y)
#         print(mal_x)
#         print(mal_y)
#         mal_x1 = inner_maximizer(
#             mal_x, mal_y, model, loss_fct, iterations=evasion_iterations, method=training_method)
#         print("THis is mal_X",mal_x1)
        # stack input
#         if is_cuda:
#             x = Variable(stack_tensors(bon_x, mal_x, mal_x1).cuda())
#             y = Variable(stack_tensors(bon_y, mal_y, mal_y).cuda())
#         else:
#             x = Variable(stack_tensors(bon_x, mal_x, mal_x1))
#             y = Variable(stack_tensors(bon_y, mal_y, mal_y))
        x = Variable(stack_tensors(bon_x, mal_x))
        y = Variable(stack_tensors(bon_y, mal_y))
#         print("STACKED")
#         print(x)
#         print(y)
#         print("This is Train Forward Pass",x, y, model )
        
        # forward pass
#         print("FORWARD X")
#         print(x)
        y_model = model(x)
#         print("YMODEL")
#         print(y_model)

        # backward pass
    # batch_idx, m, beta_type, epoch, num_epochs
        optimizer.zero_grad()
        y_model = model(x)
        kl = loss_fct(y_model, y).mean()
        print("BETA VALS")
        print(batch_idx,
                             (train_dataloader_dict["benign"].dataset.__len__() 
                                    + train_dataloader_dict["malicious"].dataset.__len__()),
                            "Soenderby",
                            epoch,
                            num_epochs)
        loss = elbo(y_model, y, kl, 
                    get_beta(batch_idx,
                             (train_dataloader_dict["benign"].dataset.__len__() 
                                    + train_dataloader_dict["malicious"].dataset.__len__()),
                            "Soenderby",
                            epoch,
                            num_epochs))
        loss.backward()
        optimizer.step()

        # predict pass
        _, predicted = torch.topk(y_model, k=1)
        correct = predicted.data.eq(y.data.view_as(predicted.data)).cpu().sum()

        # metrics
#         print("LOSS LINE 148")
#         print(len(y))
#         print(y)
#         print(total_loss)
#         print(loss.data)
#         print(loss.data.item())
        # total_loss += loss.data[0] * len(y)
        total_loss += loss.data.item() * len(y)

        total_correct += correct
        total += len(y)

        

        if 15%5 == 0:

            print("Time Taken:", time.time() - current_time)
            current_time = time.time()

            print(
                "Train Epoch ({}) | Batch ({}) | [{}/{} ({:.0f}%)]\tBatch Loss: {:.6f}\tBatch Accuracy: {:.1f}%\t BSCN: {:.12f}".
                # format(epoch, batch_idx, batch_idx * len(x),
                #        len(train_dataloader_dict["malicious"].dataset) +
                #        len(train_dataloader_dict["benign"].dataset),
                #        100. * batch_idx / len(train_dataloader_dict["benign"]), loss.data[0],
                #        100. * correct / len(y), bscn.ratio()))
                format(epoch, batch_idx, batch_idx * len(x),
                       len(train_dataloader_dict["malicious"].dataset) +
                       len(train_dataloader_dict["benign"].dataset),
                       100. * batch_idx / len(train_dataloader_dict["benign"]), loss.data.item(),
                       100. * correct / len(y), 1))


In [11]:
def check_one_category(category="benign", is_validate=False, is_evade=False,
                       evade_method='natural'):
    """
    test the model in terms of loss and accuracy on category, this function also allows to perform perturbation
    with respect to loss to evade
    :param category: benign or malicious dataset
    :param is_validate: validation or testing dataset
    :param is_evade: to perform evasion or not
    :param evade_method: evasion method (we can use on of the inner maximier methods), it is only relevant if is_evade
      is True
    :return:
    """
    model.eval()
    total_loss = 0
    total_correct = 0
    total = 0
    evasion_mode = ""

    if is_synthetic_dataset:
        # since generation of synthetic data set is random, we'd like them to be the same over epochs
        torch.manual_seed(seed_val)
        random.seed(seed_val)

    if is_validate:
        dataloader = valid_dataloader_dict[category]
    else:
        dataloader = test_dataloader_dict[category]

    for batch_idx, (x, y) in enumerate(dataloader):
        #
        if is_evade:
            x = inner_maximizer(
                x, y, model, loss_fct, iterations=evasion_iterations, method='natural')
            evasion_mode = "(evasion using %s)" % evade_method
        # stack input
        if is_cuda:
            x = Variable(x.cuda())
            y = Variable(y.cuda())
        else:
            x = Variable(x)
            y = Variable(y)
#         print("Check one CategoryForward Pass", x, y, model)
        # forward pass
        y_model = model(x)

        # loss pass
        loss = loss_fct(y_model, y).mean()

        # predict pass
        _, predicted = torch.topk(y_model, k=1)
        correct = predicted.data.eq(y.data.view_as(predicted.data)).cpu().sum()

        # metrics
        # total_loss += loss.data[0] * len(y)
        total_loss += loss.data.item() * len(y)
        total_correct += correct
        total += len(y)

    print("{} set for {} {}: Average Loss: {:.4f}, Accuracy: {:.2f}%".format(
        "Valid" if is_validate else "Test", category, evasion_mode, total_loss / total,
        total_correct * 100. / total))

    return total_loss, total_correct, total

In [12]:
def test(epoch, is_validate=False):
    """
    Function to be used for both testing and validation
    :param epoch: current epoch
    :param is_validate: is the testing done on the validation dataset
    :return: average total loss, dictionary of the metrics for both bon and mal samples
    """
    # test for accuracy and loss
    bon_total_loss, bon_total_correct, bon_total = check_one_category(
        category="benign", is_evade=False, is_validate=is_validate)
    mal_total_loss, mal_total_correct, mal_total = check_one_category(
        category="malicious", is_evade=False, is_validate=is_validate)

    # test for evasion on malicious sample
    evade_mal_total_loss, evade_mal_total_correct, evade_mal_total = check_one_category(
        category="malicious", is_evade=True, evade_method=evasion_method, is_validate=is_validate)

    total_loss = bon_total_loss + mal_total_loss
    total_correct = bon_total_correct + mal_total_correct
    total = bon_total + mal_total

    dataset_type = "valid" if is_validate else "test"

    print("{} set overall: Average Loss: {:.4f}, Accuracy: {:.2f}%".format(
        dataset_type, total_loss / total, total_correct * 100. / total))

    if is_losswise:
        graph_accuracy.append(
            epoch, {
                "%s_accuracy_%s" % (dataset_type, experiment_name): 100. * total_correct / total
            })
        graph_loss.append(epoch, {
            "%s_loss_%s" % (dataset_type, experiment_name): total_loss / total
        })
        graph_evasion.append(
            epoch, {
                "%s_evasion_%s" % (dataset_type, experiment_name):
                100 * (evade_mal_total - evade_mal_total_correct) / evade_mal_total
            })

    metrics = {
        "bscn_ratio": 0,
        "mal": {
            "total_loss": mal_total_loss,
            "total_correct": mal_total_correct,
            "total": mal_total,
            "evasion": {
                "total_loss": evade_mal_total_loss,
                "total_correct": evade_mal_total_correct,
                "total": evade_mal_total
            }
        },
        "bon": {
            "total_loss": bon_total_loss,
            "total_correct": bon_total_correct,
            "total_evade": None,
            "total": bon_total
        }
    }
    print(metrics)

    return (bon_total_loss + max(mal_total_loss, evade_mal_total_loss)) / total, metrics


In [13]:
_metrics = 0
if __name__ == "__main__":

    if not os.path.exists("result_files"):
        os.mkdir("result_files")

    _metrics = None
    session = None
    if train_model_from_scratch:
        best_valid_loss = float("inf")
        for _epoch in range(num_epochs):
            # train
            train(_epoch)

            # validate
            valid_loss, _ = test(_epoch, is_validate=True)

            # keep the best parameters w.r.t validation and check the test set
            if best_valid_loss > valid_loss:
                best_valid_loss = valid_loss
                _, _metrics = test(_epoch, is_validate=False)

                # torch.save(model, os.path.join("helper_files", "%s-model.pt" % experiment_name))
            elif _epoch % log_interval == 0:
                test(_epoch, is_validate=False)

    else:
        _, _metrics = test(0)

    # with open(os.path.join("result_files", experiment_name + ".json"), "w") as result_file:
    #     json.dump(_metrics, result_file)
    print("METRICS TO DUMP")
    print(_metrics)


dataloaderr <datasets.datasets.PortableExecutableDataset object at 0x0000021D7F64B5F8> <datasets.datasets.PortableExecutableDataset object at 0x0000021D7E6E0080>
This is bacth_idx 0
BETA VALS
0 540 Soenderby 0 10
Time Taken: 31.626336574554443
Train Epoch (0) | Batch (0) | [0/540 (0%)]	Batch Loss: 0.694362	Batch Accuracy: 50.0%	 BSCN: 1.000000000000
This is bacth_idx 1
BETA VALS
1 540 Soenderby 0 10
Time Taken: 0.025926828384399414
Train Epoch (0) | Batch (1) | [32/540 (6%)]	Batch Loss: 0.682474	Batch Accuracy: 62.5%	 BSCN: 1.000000000000
This is bacth_idx 2
BETA VALS
2 540 Soenderby 0 10
Time Taken: 0.026927947998046875
Train Epoch (0) | Batch (2) | [64/540 (12%)]	Batch Loss: 3.347756	Batch Accuracy: 50.0%	 BSCN: 1.000000000000
This is bacth_idx 3
BETA VALS
3 540 Soenderby 0 10
Time Taken: 0.026928186416625977
Train Epoch (0) | Batch (3) | [96/540 (18%)]	Batch Loss: 0.818466	Batch Accuracy: 50.0%	 BSCN: 1.000000000000
This is bacth_idx 4
BETA VALS
4 540 Soenderby 0 10
Time Taken: 0.02

Valid set for malicious (evasion using natural): Average Loss: 0.2746, Accuracy: 94.44%
valid set overall: Average Loss: 0.3229, Accuracy: 88.89%
{'bscn_ratio': 0, 'mal': {'total_loss': 24.714837610721588, 'total_correct': tensor(85), 'total': 90, 'evasion': {'total_loss': 24.71483725309372, 'total_correct': tensor(85), 'total': 90}}, 'bon': {'total_loss': 33.40518367290497, 'total_correct': tensor(75), 'total_evade': None, 'total': 90}}
Test set for benign : Average Loss: 0.2692, Accuracy: 88.89%
Test set for malicious : Average Loss: 0.3402, Accuracy: 91.11%
Test set for malicious (evasion using natural): Average Loss: 0.3402, Accuracy: 91.11%
test set overall: Average Loss: 0.3047, Accuracy: 90.00%
{'bscn_ratio': 0, 'mal': {'total_loss': 30.61383393406868, 'total_correct': tensor(82), 'total': 90, 'evasion': {'total_loss': 30.61383393406868, 'total_correct': tensor(82), 'total': 90}}, 'bon': {'total_loss': 24.22561740875244, 'total_correct': tensor(80), 'total_evade': None, 'total':

Valid set for benign : Average Loss: 0.7738, Accuracy: 87.78%
Valid set for malicious : Average Loss: 0.6050, Accuracy: 86.67%
Valid set for malicious (evasion using natural): Average Loss: 0.6050, Accuracy: 86.67%
valid set overall: Average Loss: 0.6894, Accuracy: 87.22%
{'bscn_ratio': 0, 'mal': {'total_loss': 54.454221963882446, 'total_correct': tensor(78), 'total': 90, 'evasion': {'total_loss': 54.454222202301025, 'total_correct': tensor(78), 'total': 90}}, 'bon': {'total_loss': 69.63866859674454, 'total_correct': tensor(79), 'total_evade': None, 'total': 90}}
dataloaderr <datasets.datasets.PortableExecutableDataset object at 0x0000021D7F64B5F8> <datasets.datasets.PortableExecutableDataset object at 0x0000021D7E6E0080>
This is bacth_idx 0
BETA VALS
0 540 Soenderby 4 10
Time Taken: 30.361685037612915
Train Epoch (4) | Batch (0) | [0/540 (0%)]	Batch Loss: 0.146860	Batch Accuracy: 96.9%	 BSCN: 1.000000000000
This is bacth_idx 1
BETA VALS
1 540 Soenderby 4 10
Time Taken: 0.0269269943237

Valid set for malicious (evasion using natural): Average Loss: 0.9248, Accuracy: 95.56%
valid set overall: Average Loss: 1.5585, Accuracy: 90.00%
{'bscn_ratio': 0, 'mal': {'total_loss': 83.23153793811798, 'total_correct': tensor(86), 'total': 90, 'evasion': {'total_loss': 83.23153564333916, 'total_correct': tensor(86), 'total': 90}}, 'bon': {'total_loss': 197.30576992034912, 'total_correct': tensor(76), 'total_evade': None, 'total': 90}}
dataloaderr <datasets.datasets.PortableExecutableDataset object at 0x0000021D7F64B5F8> <datasets.datasets.PortableExecutableDataset object at 0x0000021D7E6E0080>
This is bacth_idx 0
BETA VALS
0 540 Soenderby 6 10
Time Taken: 31.707828998565674
Train Epoch (6) | Batch (0) | [0/540 (0%)]	Batch Loss: 0.000089	Batch Accuracy: 100.0%	 BSCN: 1.000000000000
This is bacth_idx 1
BETA VALS
1 540 Soenderby 6 10
Time Taken: 0.027925729751586914
Train Epoch (6) | Batch (1) | [32/540 (6%)]	Batch Loss: 0.061795	Batch Accuracy: 96.9%	 BSCN: 1.000000000000
This is bact

This is bacth_idx 0
BETA VALS
0 540 Soenderby 8 10
Time Taken: 30.608741521835327
Train Epoch (8) | Batch (0) | [0/540 (0%)]	Batch Loss: 0.069835	Batch Accuracy: 96.9%	 BSCN: 1.000000000000
This is bacth_idx 1
BETA VALS
1 540 Soenderby 8 10
Time Taken: 0.026926279067993164
Train Epoch (8) | Batch (1) | [32/540 (6%)]	Batch Loss: 0.114548	Batch Accuracy: 93.8%	 BSCN: 1.000000000000
This is bacth_idx 2
BETA VALS
2 540 Soenderby 8 10
Time Taken: 0.029920101165771484
Train Epoch (8) | Batch (2) | [64/540 (12%)]	Batch Loss: 0.063989	Batch Accuracy: 96.9%	 BSCN: 1.000000000000
This is bacth_idx 3
BETA VALS
3 540 Soenderby 8 10
Time Taken: 0.029947519302368164
Train Epoch (8) | Batch (3) | [96/540 (18%)]	Batch Loss: 0.905784	Batch Accuracy: 96.9%	 BSCN: 1.000000000000
This is bacth_idx 4
BETA VALS
4 540 Soenderby 8 10
Time Taken: 0.018921852111816406
Train Epoch (8) | Batch (4) | [128/540 (24%)]	Batch Loss: 0.027368	Batch Accuracy: 100.0%	 BSCN: 1.000000000000
This is bacth_idx 5
BETA VALS
5 5

In [14]:
type(_metrics)

dict

In [15]:
_metrics

{'bscn_ratio': 0,
 'mal': {'total_loss': 30.61383393406868,
  'total_correct': tensor(82),
  'total': 90,
  'evasion': {'total_loss': 30.61383393406868,
   'total_correct': tensor(82),
   'total': 90}},
 'bon': {'total_loss': 24.22561740875244,
  'total_correct': tensor(80),
  'total_evade': None,
  'total': 90}}

In [16]:
_metrics['mal']['total']

90

In [17]:
def confMat(_metrics):
    total_mal = _metrics['mal']['total']
    total_mal_correct = _metrics['mal']['total_correct']
    total_mal_wrong = total_mal - total_mal_correct
    
    total_ben = _metrics['bon']['total']
    total_ben_correct = _metrics['bon']['total_correct']
    total_ben_wrong = total_ben - total_ben_correct
    
    true_list = []
    for i in range(total_mal):
        true_list.append(1)
    for i in range(total_ben):
        true_list.append(0)
        
    pred_list = []
    for i in range(total_mal_correct):
        pred_list.append(1)
    for i in range(total_mal_wrong):
        pred_list.append(0)
    for i in range(total_ben_correct):
        pred_list.append(0)
    for i in range(total_ben_wrong):
        pred_list.append(1)
    return true_list,pred_list

In [18]:
TrueList,PredList = confMat(_metrics)

In [19]:
np.unique(np.array(TrueList),return_counts=True)

(array([0, 1]), array([90, 90], dtype=int64))

In [20]:
np.unique(np.array(PredList),return_counts=True)

(array([0, 1]), array([88, 92], dtype=int64))

In [21]:
from sklearn import metrics

In [22]:
print(metrics.confusion_matrix(TrueList,PredList))

[[80 10]
 [ 8 82]]


In [23]:
print(metrics.classification_report(TrueList,PredList))

              precision    recall  f1-score   support

           0       0.91      0.89      0.90        90
           1       0.89      0.91      0.90        90

    accuracy                           0.90       180
   macro avg       0.90      0.90      0.90       180
weighted avg       0.90      0.90      0.90       180

